# 3. Библиотеки анализа данных в Python. Pandas

**Pandas** - это библиотека Python, предоставляющая широкие возможности для анализа данных. С ее помощью очень удобно загружать, обрабатывать и анализировать табличные данные с помощью SQL-подобных запросов. В связке с библиотеками Matplotlib и Seaborn появляется возможность удобного визуального анализа табличных данных.

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Основными структурами данных в Pandas являются классы `Series` и `DataFrame`. Первый из них представляет собой одномерный индексированный массив данных некоторого фиксированного типа. Второй - это двухмерная структура данных, представляющая собой таблицу, каждый столбец которой содержит данные одного типа. Можно представлять её как словарь объектов типа `Series`. Структура `DataFrame` отлично подходит для представления реальных данных: строки соответствуют признаковым описаниям отдельных объектов, а столбцы соответствуют признакам.

Для начала рассмотрим простые примеры создания таких объектов и возможных операций над ними.

## Series

Создание объекта `Series` из 5 элементов, индексированных буквами:

In [17]:
s = pd.Series(np.random.randn(5), 
              index = ['a', 'b', 'c', 'd', 'e']) 
print(s)                                                                 

a   -0.527938
b   -0.661846
c    0.665612
d   -2.214692
e    0.402283
dtype: float64


Индексирование возможно в виде `s.Name` или `s['Name']`.

In [18]:
print(s.b == s['b'])

True


`Series` поддерживает пропуски в данных.

In [19]:
s.c = np.nan # Series can contain missing values
print(s)

a   -0.527938
b   -0.661846
c         NaN
d   -2.214692
e    0.402283
dtype: float64


Объекты `Series` похожи на обычный список и могут быть переданы в качестве аргументов большинству функций из NumPy.

In [20]:
print('Second element of s is', s[1], '\n')
# Smart indexing
print(s[:3], '\n')
print('There are', len(s[s > 0]), 'positive elements in s\n')
# Series obects can be the arguments for Numpy functions
print(np.exp(s))

Second element of s is -0.6618455305001097 

a   -0.527938
b   -0.661846
c         NaN
dtype: float64 

There are 1 positive elements in s

a    0.589820
b    0.515898
c         NaN
d    0.109187
e    1.495235
dtype: float64


## DataFrame

### Создание и изменение

Перейдём к рассмотрению объектов типа `DataFrame`. Такой объект можно создать из массива numpy, указав названия строк и столбцов.

In [21]:
df1 = pd.DataFrame(np.random.randn(5, 3), 
                   index=['o1', 'o2', 'o3', 'o4', 'o5'], 
                   columns=['f1', 'f2', 'f3'])
df1

,f1,f2,f3
o1,0.730244,0.855091,-0.936119
o2,0.649504,0.109951,1.205281
o3,-0.303615,-0.737846,0.240819
o4,0.653282,-0.059404,-0.106795
o5,-0.482359,-0.127208,0.428202


Альтернативным способом является создание `DataFrame` из словаря numpy массивов или списков.

In [24]:
df2 = pd.DataFrame({'A': np.random.random(5), 
                    'B': ['a', 'b', 'c', 'd', 'e'], 
                    'C': np.arange(5) > 2})
df2

,A,B,C
0,0.337790,a,False
1,0.043377,b,False
2,0.510169,c,False
3,0.755182,d,True
4,0.905834,e,True


Обращение к элементам (или целым кускам фрейма):

In [25]:
print('The element in position 3, B is', df2.at[3, 'B'], '\n')
print(df2.loc[[1, 4], ['A', 'B']])

The element in position 3, B is d 

          A  B
1  0.043377  b
4  0.905834  e


Изменение элементов и добавление новых:

In [26]:
df2.at[2, 'B'] = 'f'
df2

,A,B,C
0,0.337790,a,False
1,0.043377,b,False
2,0.510169,f,False
3,0.755182,d,True
4,0.905834,e,True


In [27]:
df2.loc[5] = [3.1415, 'c', False]
df2

,A,B,C
0,0.337790,a,False
1,0.043377,b,False
2,0.510169,f,False
3,0.755182,d,True
4,0.905834,e,True
5,3.141500,c,False


In [28]:
df1.columns = ['A', 'B', 'C']
df3 = df1.append(df2)
df3

,A,B,C
o1,0.730244,0.855091,-0.936119
o2,0.649504,0.109951,1.205281
o3,-0.303615,-0.737846,0.240819
o4,0.653282,-0.059404,-0.106795
o5,-0.482359,-0.127208,0.428202
0,0.337790,a,0.000000
1,0.043377,b,0.000000
2,0.510169,f,0.000000
3,0.755182,d,1.000000
4,0.905834,e,1.000000


#### Обработка пропущенных значений

In [29]:
df1.at['o2', 'A'] = np.nan
df1.at['o4', 'C'] = np.nan
df1

,A,B,C
o1,0.730244,0.855091,-0.936119
o2,NaN,0.109951,1.205281
o3,-0.303615,-0.737846,0.240819
o4,0.653282,-0.059404,NaN
o5,-0.482359,-0.127208,0.428202


Булева маска для пропущенных значений (True - там, где был пропуск, иначе - False):

In [30]:
pd.isnull(df1)

,A,B,C
o1,False,False,False
o2,True,False,False
o3,False,False,False
o4,False,False,True
o5,False,False,False


Можно удалить все строки, где есть хотя бы один пропуск.

In [31]:
df1.dropna(how='any')

,A,B,C
o1,0.730244,0.855091,-0.936119
o3,-0.303615,-0.737846,0.240819
o5,-0.482359,-0.127208,0.428202


Пропуски можно заменить каким-то значением.

In [15]:
df1.fillna(0)

,A,B,C
o1,-0.352231,-1.416276,2.549509
o2,0.000000,0.644604,1.387981
o3,-0.256846,-1.146433,0.188870
o4,-0.901309,0.065731,0.000000
o5,1.360262,-0.824708,0.015757
